Basic routines in order to retrieve information from multiple chemical databases.

In [23]:
import libchebipy
import pubchempy
from Bio.KEGG import REST as krest
from rdkit import Chem

def chinfo(cmpd):
    chebi = libchebipy.ChebiEntity(cmpd)
    return( chebi.get_name(), chebi.get_inchi(), chebi.get_smiles() )
def pbinfo(cmpd):
    cpd = pubchempy.Compound.from_cid( cmpd )
    return( cpd.synonyms[0], cpd.inchi, cpd.canonical_smiles )
def keinfo(cmpd):
    info = krest.kegg_get( cmpd )
    chebi = None
    for line in info:
        if line.startswith('NAME'):
            name = line.rstrip()[12:].rstrip(';')
        if line[12:(12+len('ChEBI'))] == 'ChEBI':
            chebi = line[12:].split(' ')[1]
            return chinfo( chebi ) + (chebi,)
    info = krest.kegg_get( cmpd, 'mol' )
    mol = Chem.MolFromMolBlock( info.read() )
    return( name, Chem.MolToInchi(mol), Chem.MolToSmiles(mol), chebi )

cmpd = 'CHEBI:27452'
chinfo(cmpd)
cmpd = '7501'
pbinfo(cmpd)
cmpd = 'C07083'
keinfo(cmpd)


('styrene',
 'InChI=1S/C8H8/c1-2-8-6-4-3-5-7-8/h2-7H,1H2',
 'C=Cc1ccccc1',
 '27452\n')

In [ ]:
import pandas as pd
import re, os, glob

for f in glob.glob('*_reactions.xlsx'):
    dfd = pd.read_excel(f, sheet_name=None)
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    fn = os.path.join('annotated', f)
    writer = pd.ExcelWriter(fn, engine='xlsxwriter')
    for s in dfd:
        df = dfd[s]
        df['dbName'] = 'na'
        for i in df.index:
            try:
                info = None
                if re.search('chebi', df.loc[i,'ID']):
                    m = df.loc[i,'ID'].split(':')
                    info = chinfo(m[-1]) 
                elif re.search('kegg', df.loc[i,'ID']):
                    m = df.loc[i,'ID'].split(':')
                    info = keinfo(m[-1])
                    chebi = info[3]
                    if info[3] is not None:
                        df.loc[i,'ID'] = 'http://identifiers.org/chebi/CHEBI:'+info[3]
                elif re.search('pubchem', df.loc[i,'ID']):
                    m = df.loc[i,'ID'].split('/')
                    info = pbinfo(m[-1])
                if info is not None:
                    df.loc[i,'dbName'] = info[0]
                    df.loc[i,'InChI'] = info[1]
                    df.loc[i,'SMILES'] = info[2]
            except:
                if df.loc[i,'Role'] != 'Reaction':
                    print(df.iloc[i,:])
                continue
        df.to_excel(writer, sheet_name=s, index=False)
    writer.save()